The standard implementation of python is called **CPython**. 

CPython runs a python program in two steps:
    * parses and compiles the source text code into bytecode
    * runs the bytecode using a stack-based interpreter
    
The bytecode interpreter has state that must be maintained and coherent while the python program executes. Python enforces coherence with a mechanism called the *global interpreter lock (GIL)*.

**GIL**: is a mutual-exclusion lock that prevents CPython from bein affected by preemptive multithreading = one thread takes control of a program by interrupting another thread. Such an interruption could corrupt the interpreter state if it comes at an unexpected time. The GIL prevents these interruptions and ensures that every bytecode instruction works correctly.

[For more infos about GIL](https://wiki.python.org/moin/GlobalInterpreterLock)

__Negative side effect:__

Even if Python supports multiple threads of execution, the GIL causes only one of them to make forward progress at a time. 

This means that when you reach for threads to do parallel computation and speed up our Python code, that will not work as expected !

### Example

In [1]:
def factorize(number):
    for i in range(1, number+1):
        if number % i == 0:
            yield i

In [2]:
import time

In [3]:
numbers = [2139079, 1214759, 1516637, 1582285]
start = time.time()
for number in numbers:
    list(factorize(number))
end = time.time()
print("It took %.3f seconds" % (end - start))

It took 0.699 seconds


Let's try to use multithreading for this !

In [4]:
from threading import Thread

class FactorizeThread(Thread):
    
    def __init__(self, number):
        super().__init__()
        self.number = number
        
    def run(self):
        self.factors = list(factorize(self.number))

In [6]:
start = time.time()
threads = []
for number in numbers:
    thread = FactorizeThread(number)
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

end = time.time()
print("Took %.3f seconds" % (end-start))

Took 0.854 seconds


Suprisingly, it took longer !!!!

This illustrate the effect of the GIL on programs running in the standard CPython interpreter.

There are ways to get CPython to utilize multiple cores, but that does not work with the standard **Thread** class. 

### Thread in Python

* with threads, we can leave it to Python to run functions seemingly in parallel.
* blocking I/O. This happens when Python does certain types of system calls. Using thread helps us handle blocking I/O by insulating our program from the time it takes for the OS to respond to our request.

    for example, if we want to control a remote airplane, we want to send a command but in the same time listen to another one....etc, they have to be in parallel.